In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from csv import reader
import copy 
import cv2
from mpl_toolkits import mplot3d
import argparse
import math


In [2]:
def cleanUp(directory):
    ''' Set outliers to threshold value in csv-format data from SPIM-FRAP'd cell nuclei. In the original use of this
    code, an outlier-riddled file named '0CRECO_200731.csv' was converted to a clean file called 'C0R.csv'. '''
    
    objlist = []
    for filename in os.listdir(directory + '/unclean data'):
        if filename.endswith(".csv"):

            cellnum = filename[1] 
            compstate = filename[0]
            datatype = filename[2]
            objname = compstate + cellnum + datatype
            objlist.append(objname)
            
            
            # Convert to numpy array so values can be modified and analyzed
            a = np.array(pd.read_csv(directory + '/unclean data' + '/{}'.format(filename))) 
            a0 = copy.deepcopy(a) 
            
            # Accumulate individual nucleus data points in a list
            vallist = [] 
            vallist.append(a[a != 0.0])
            
            # Establish outlier qualifications
            Q1 = np.percentile(vallist, 25)
            Q2 = np.percentile(vallist, 50)
            Q3 = np.percentile(vallist, 75)
            UPPERTHRESH = Q3 + ((Q3-Q1)*1.5)
            if (datatype == 'r') and (UPPERTHRESH > 1.0):       # recovery percent values should be less than 1.0
                UPPERTHRESH = 1.0
            LOWERTHRESH = Q1 - ((Q3-Q1)*1.5)
            if LOWERTHRESH < 0.0:                               # no values should be negative
                LOWERTHRESH = 0.0
            
            # Set outliers to the threshold value
            a[a > UPPERTHRESH] = UPPERTHRESH
            a[(a < LOWERTHRESH) & (a != 0.0)] = LOWERTHRESH

            # Verify whether any outliers exist and if they ever existed
            if np.any(a > UPPERTHRESH):                                 
                print('ERROR: high outliers remain in', filename, 'threshold: ', Q3)
            if np.any((a != 0.0) & (a < LOWERTHRESH)):
                print('ERROR: low outliers remain in', filename, 'threshold: ', Q1)
            elif np.any(a - a0 != 0) == True:                                   
                print(filename, ' has been cleaned up')
                pd.DataFrame(a).to_csv(directory + '/cleaned data' + 
                                       '/{}'.format(objname) + '.csv', header=None, index=None) 
            else:
                print(filename, ' had no outliers')
                pd.DataFrame(a).to_csv(directory + '/cleaned data' + 
                                       '/{}'.format(objname) + '.csv', header=None, index=None)
                

In [3]:
def SOBEL(cs, cn, dt, kernel, orientation):
    ''' Visualize the approximation of the gradient value at each location in the cell nucleus. This should indicate 
    the extent to which neighboring recovery percentage or recovery time values are changing at each point, and give a
    measure of texture in the image. Orientation 'X' will allow you to see your kernel convolved horizontally, orientation
    'Y' will show the vertical convolution results, and orientation 'XY' combines these images and is what I have found 
    most useful to work with. Your designated kernel size will allow you to choose how many neighboring pixels to
    incorporate into this measurement, and must be an odd integer. A larger value for this may be less useful in
    considering small cells, as you may amplify the contributions of the nuclear boundary (where there is the most
    sudden increase or decrease in intensity) throughout the file. This is an existing function, part of the CV2 library. '''
    
    b = np.array(pd.read_csv(directory + '/cleaned data/' + cs + cn + dt + '.csv'))
    matrixX = cv2.Sobel(b, cv2.CV_64F, 1, 0, ksize=kernel)
    matrixY = cv2.Sobel(b, cv2.CV_64F, 0, 1, ksize=kernel)
    matrixXY = np.sqrt((matrixX**2) + (matrixY**2))
    
    if orientation == "X":
        return matrixX
    if orientation == "Y":
        return matrixY
    if orientation == "XY":
        pd.DataFrame(matrixXY).to_csv(directory + '/sobel/' + cs + cn + dt + '.csv', header=None, index=None)
        return matrixXY
    
    plt.subplot(2,2,1),plt.imshow(b, cmap = 'gray')
    plt.title('Original File'), plt.xticks([]), plt.yticks([])
    plt.subplot(2,2,2),plt.imshow(matrixXY, cmap = 'gray')
    plt.title('Combined Sobel'), plt.xticks([]), plt.yticks([])
    plt.subplot(2,2,3),plt.imshow(matrixX,cmap = 'gray')
    plt.title('Sobel X'), plt.xticks([]), plt.yticks([])
    plt.subplot(2,2,4),plt.imshow(matrixY,cmap = 'gray')
    plt.title('Sobel Y'), plt.xticks([]), plt.yticks([])
    plt.show()
    

In [4]:
def eatEdge(compstate, cellnum, datatype, directory):
    ''' Eat away the top and bottom edge of a sobel-processed SPIM-FRAP'd cell image so that texture values 
    included do not represent the boundary of the cell. In the original use of this code function, COMPSTATE = 'u' or 'c',
    CELLNUM is an integer referring to a cell in the data set (0-6), and DATATYPE = 'r' or 't'. It is very important that
    you visualize the results of this process, which can be a little less reliable when employing larger values for
    the kernel size of the sobel operator. If needed, the manualCorrection factor can be used to adjust the final
    iteration count as you see fit. '''
    
    
    # Load file into matrix, set labelled indices to 0.0
    s = np.array(pd.read_csv(directory + '/sobel/' + compstate + cellnum + datatype + '.csv')) 
    s[0,:] = 0.0
    s[:,0] = 0.0
    
    s0 = copy.deepcopy(s)   # create copy to compare result against
    t = copy.deepcopy(s)    # create copy to eat once appropriate iteration value is established
    
    
    # Instantiate lists
    sumList = [np.sum(s)]   # contain sum values for each eating trial 
    tumList = [np.sum(t)]
    ratioList = []          # compare one value in sumList to the previous
    tRatioList = []
    
    
    # Establish appropriate number of times to eat edges
    for iter in range(20):                              # try eating the edges 20 times
        for i in range(1, s.shape[1]):                  
            if np.any(s[:,i] != 0.0):                   
                jmin = np.min(np.where(s[:,i] > 0.0))   # find top edge
                jmax = np.max(np.where(s[:,i] > 0.0))   # find bottom edge
                s[jmin,i] = 0.0                         # EAT TOP EDGE
                s[jmax,i] = 0.0                         # EAT BOTTOM EDGE
        sumList.append(np.sum(s))                       # record the sum of all values in current matrix
        ratioList.append(sumList[-1]/sumList[-2])
    
    
    # Find how many trials it took to achieve the least significant decrease
    eatTrials = np.argmax(ratioList[3:])                # only measurements made after >=3 iterations are viable
    print('eatTrials = ', eatTrials)
    
    manualCorrection = 0                                # increase or decrease number if you suspect that more or less iterations are needed
    print('manualCorrection = ', manualCorrection)
    
    # Repeat the eating sequence on copied matrix appropriately
    for iter in range(eatTrials + 2 + manualCorrection):                  
        for i in range(1, t.shape[1]):                  
            if np.any(t[:,i] != 0.0):                   
                jmin = np.min(np.where(t[:,i] > 0.0))  
                jmax = np.max(np.where(t[:,i] > 0.0)) 
                t[jmin,i] = 0.0                           
                t[jmax,i] = 0.0
        tumList.append(np.sum(t))
        tRatioList.append(tumList[-1]/tumList[-2])
    
    
    # Save the new, edgeless file
    pd.DataFrame(t).to_csv(directory + '/sobel no edge/' + compstate + cellnum + datatype + '.csv',
                              header=None, index=None)        

    # See if the results look reasonable
    plt.subplot(2,2,1), plt.imshow(s0, cmap = 'gray')
    plt.title('original file')
    plt.subplot(2,2,2), plt.imshow(t, cmap = 'gray')
    plt.title('edgeless file')
    

In [5]:
def indentionIdentifier(cellnum, datatype, directory, step):
    ''' Identify the region of a cell nucleus that has been compressed by an AFM cantilever. Function returns
    two indices, representing the columns that you should limit your examinations to when studying the compressed
    region or the outside region. You can control how narrow this region is by specifying the 'step', which should
    be on the order of 10 pixels, and allows you to step inward (wilth positive number) or outward (with negaitive value)
    from each of the bumps that are identified as the points where the compressed cell is tallest. '''
    
    
    # Load the confined and not confined data for the same cell nucleus
    c = np.array(pd.read_csv(directory + '/sobel no edge/' + 'c' + cellnum + datatype + '.csv')) 
    u = np.array(pd.read_csv(directory + '/sobel no edge/' + 'u' + cellnum + datatype + '.csv'))
    
    
    # Find the length column that contains data for the nucleus
    vertLength = []            # length of vertical strips for compressed cell
    uLength = []               # length of vertical strips for noncompressed cell
    columns = []               # associated column indices
    
    
    for i in range(1, c.shape[1]):                     # look at each column
        columns.append(i)                              # record column index
        
        if np.any( c[:, i] != 0.0 ):
            j0 = np.min(np.where( c[:,i] > 0.0 ))
            jf = np.max(np.where( c[:,i] > 0.0 ))
            vL = jf - j0                               # record length of column in compressed cell data
            vertLength.append(vL)
        if np.all( c[:, i] == 0.0 ):
            vertLength.append(0.0)
                
            
        if np.any( u[:, i] != 0.0 ):
            uj0 = np.min(np.where( u[:,i] > 0.0 ))
            ujf = np.max(np.where( u[:,i] > 0.0 ))
            uvL = ujf - uj0                            # record length of column from non-compressed cell data
            uLength.append(uvL)
        if np.all( u[:,i] == 0.0 ):
            uLength.append(0.0)
            
            
    # Examine the change in the height of the cell nucleus to place bump indices
    dVL = np.diff(vertLength)
    mp = int(0.5*(columns[0] + columns[-1]))
    
    rightBump = 0
    for n in range(1, mp):
        rightBump = np.max(np.where(dVL > 0.0))
    leftBump = 0
    for m in range(mp, len(columns)-2):
        leftBump = np.min(np.where(dVL < 0.0))
        
        
    # Uncomment the following code to compare the lengths of the data-populated regions in each column amond files (u vs. c)
    #for i in range(len(vertLength)):
    #    print(str(vertLength[i+1]) + '\t' +  str(dVL[i]) + '\t' + str(uLength[i+1]) + '\t' + str(columns[i]))
    
    return [leftBump + step, rightBump - step]


In [6]:
def dataGet(cellnum, datatype, directory, step, Type):
    ''' Compare the Median or the Mean of either the compressed regions or the outer regions between data for a 
    cell measured in and before confinement. Visualize the region from which you are measuring a value, so that
    you can adjust your step size if necessary. '''
    
    c = np.array(pd.read_csv(direct + '/sobel no edge/' + 'c' + cellnum + datatype + '.csv')) 
    u = np.array(pd.read_csv(direct + '/sobel no edge/' + 'u' + cellnum + datatype + '.csv'))
    c0 = copy.deepcopy(c)
    u0 = copy.deepcopy(u)
    
    [L,R] = indentionIdentifier(cellnum, datatype, directory, step)
    
    noncData = 0.0
    compData = 0.0
    
    if Type == 'InnerMean':
        u0[:,:(L-1)] = 0.0
        u0[:,(R+1):] = 0.0
        c0[:,:(L-1)] = 0.0
        c0[:,(R+1):] = 0.0
        noncData = np.average(np.nonzero(u0))
        compData = np.average(np.nonzero(c0))
    
    if Type == 'OuterMean':
        u0[:,L:R] = 0.0
        c0[:,L:R] = 0.0
        noncData = np.average(np.nonzero(u0))
        compData = np.average(np.nonzero(c0))
        
    if Type == 'InnerMedian':
        u0[:,:(L-1)] = 0.0
        u0[:,(R+1):] = 0.0
        c0[:,:(L-1)] = 0.0
        c0[:,(R+1):] = 0.0
        noncData = np.median(np.nonzero(u0))
        compData = np.median(np.nonzero(c0))
        
    if Type == 'OuterMedian':
        u0[:,L:R] = 0.0
        c0[:,L:R] = 0.0
        noncData = np.median(np.nonzero(u0))
        compData = np.median(np.nonzero(c0))

        
    plt.subplot(2,2,1),plt.imshow(u,cmap='gray')
    plt.title('non-compressed cell '+ cellnum)
    plt.subplot(2,2,2),plt.imshow(c,cmap='gray')
    plt.title('compressed cell '+ cellnum)
    plt.subplot(2,2,3),plt.imshow(u0,cmap='gray')
    plt.title(str(noncData))
    plt.subplot(2,2,4),plt.imshow(c0,cmap='gray')
    plt.title(str(compData))
        
    return noncData, compData
